In [ ]:
!pip install pyxlsb
!pip install pandas_market_calendars
!pip install stable-baselines3
!pip install shimmy>=0.2.1
!pip install keras-rl2

In [ ]:
from google.colab import drive
import pandas as pd
from pyxlsb import open_workbook

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Pré-Processamento

In [ ]:
# Caminho para o arquivo xlsb no Google Drive
file_path = "../data/"

df_purchase = pd.read_excel(file_path + "base2023_compra.xlsb", engine="pyxlsb")
df_purchase_2 = pd.read_excel(file_path + "base2023_compra_2.xlsb", engine="pyxlsb")

In [ ]:
# change the date columns to pandas datetime
df_purchase_datetime = df_purchase.copy()
df_purchase_2_datetime = df_purchase_2.copy()

df_purchase_datetime['Dt. Operação'] = pd.to_datetime(df_purchase_datetime['Dt. Operação'], unit='D', origin='1899-12-30')
df_purchase_2_datetime['Dt. Operação'] = pd.to_datetime(df_purchase_2_datetime['Dt. Operação'], format="%d/%m/%Y")

df_purchase_2_datetime.rename(columns={"Vencimento": "Dt. Liquidação"}, inplace=True)

df_purchase_datetime['Dt. Liquidação'] = pd.to_datetime(df_purchase_datetime['Dt. Liquidação'], unit='D', origin='1899-12-30')
df_purchase_2_datetime['Dt. Liquidação'] = pd.to_datetime(df_purchase_2_datetime['Dt. Liquidação'], errors='coerce', format="%d/%m/%Y")

df_purchases = pd.concat([df_purchase_datetime, df_purchase_2_datetime], ignore_index=True)
df_purchases = df_purchases.sort_values(by=["Cód. Cliente", "Dt. Operação", "Cód. Título", "Cód. Corretora"]).reset_index()
pd.set_option('display.max_columns', None)

In [ ]:
df_sales = pd.read_excel(file_path + "base2023_venda.xlsb", engine="pyxlsb")

In [ ]:
df_sales_datetime = df_sales.copy()
df_sales_datetime['Dt. Operação'] = pd.to_datetime(df_sales_datetime['Dt. Operação'], unit='D', origin='1899-12-30')
df_sales_datetime['Vencimento'] = pd.to_datetime(df_sales_datetime['Vencimento'], unit='D', origin='1899-12-30')
df_sales_datetime = df_sales_datetime.sort_values(by=["Cód. Cliente", "Dt. Operação", "Cód. Título", "Cód. Corretora"]).reset_index()

In [ ]:
display(df_sales_datetime.head())

,index,Chave,Cód. Cliente,Dt. Operação,Tipo Operação,Cód. Título,Cód. Corretora,Quantidade,Preço,Valor Líquido,Vencimento,Tipo,DI
0,4720,4721A,Cliente 1,2023-01-05,V,XPBR31,XPIN,4470,76.450544,341733.93,2023-03-21,TE,0.136781
1,19006,19007A,Cliente 1,2023-01-05,V,XPBR31,XPIN,35530,76.440559,2715933.07,2023-03-21,TE,0.136781
2,20954,20955A,Cliente 1,2023-01-05,V,XPBR31,XPIN,5440,76.069360,413817.32,2023-03-06,TE,0.136704
3,30030,30031A,Cliente 1,2023-01-05,V,XPBR31,XPIN,14560,76.079336,1107715.13,2023-03-06,TE,0.136704
4,18898,18899A,Cliente 1,2023-01-19,V,XPBR31,XPIN,41230,86.807599,3579077.32,2023-02-22,TE,0.136585


In [ ]:
display(df_purchases.head())

,index,Cód. Cliente,Dt. Operação,Tipo Operação,Cód. Título,Cód. Corretora,Quantidade,Preço,Valor Líquido,Dt. Liquidação,Tipo
0,612315,Cliente 1,2023-01-02,C,ABEV3,FLOW,162,14.203951,-2301.04,2023-01-04,AV
1,614574,Cliente 1,2023-01-02,C,ALPA4,CONV,1070,14.340467,-15344.30,2023-01-04,AV
2,448479,Cliente 1,2023-01-02,C,ALSO3,FLOW,63,16.296032,-1026.65,2023-01-04,AV
3,697579,Cliente 1,2023-01-02,C,ALUP11,FLOW,48,27.697500,-1329.48,2023-01-04,AV
4,603010,Cliente 1,2023-01-02,C,AMER3,CONV,877,9.036602,-7925.10,2023-01-04,AV


In [ ]:
df_filtered_purchases = df_purchases.copy()
df_filtered_sales = df_sales_datetime.copy()

# removing unused columns
df_filtered_purchases = df_filtered_purchases.drop(["index", "Tipo Operação", "Valor Líquido", "Dt. Liquidação", "Tipo"], axis=1)
df_filtered_sales = df_filtered_sales.drop(["index", "Tipo Operação", "Valor Líquido", "Tipo"], axis=1)

In [ ]:
import pandas_market_calendars as mcal
brazil_calendar = mcal.get_calendar('BMF')

def calculate_du(x):
  """Calculates the number of business days between the sale operation date and the expiration date

  Args:
      x (dataframe): dataframe with the sales operations

  Returns:
      int: number of business days
  """
  business_days = brazil_calendar.valid_days(x["Dt. Operação"], x["Vencimento"])
  du = len(business_days) - 1
  return du

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

# apply the calculate_du function to each row of the dataframe
df_filtered_sales["du"] = df_filtered_sales.apply(calculate_du, axis=1)

In [ ]:
display(df_filtered_purchases.head())
display(df_filtered_sales.head())

,Cód. Cliente,Dt. Operação,Cód. Título,Cód. Corretora,Quantidade,Preço
0,Cliente 1,2023-01-02,ABEV3,FLOW,162,14.203951
1,Cliente 1,2023-01-02,ALPA4,CONV,1070,14.340467
2,Cliente 1,2023-01-02,ALSO3,FLOW,63,16.296032
3,Cliente 1,2023-01-02,ALUP11,FLOW,48,27.697500
4,Cliente 1,2023-01-02,AMER3,CONV,877,9.036602


,Chave,Cód. Cliente,Dt. Operação,Cód. Título,Cód. Corretora,Quantidade,Preço,Vencimento,DI,du
0,4721A,Cliente 1,2023-01-05,XPBR31,XPIN,4470,76.450544,2023-03-21,0.136781,51
1,19007A,Cliente 1,2023-01-05,XPBR31,XPIN,35530,76.440559,2023-03-21,0.136781,51
2,20955A,Cliente 1,2023-01-05,XPBR31,XPIN,5440,76.069360,2023-03-06,0.136704,40
3,30031A,Cliente 1,2023-01-05,XPBR31,XPIN,14560,76.079336,2023-03-06,0.136704,40
4,18899A,Cliente 1,2023-01-19,XPBR31,XPIN,41230,86.807599,2023-02-22,0.136585,22


In [ ]:
df_filtered_sales_du = df_filtered_sales.copy()
# filter the sales dataframe to only include sales with a du between 1 and 252 (one year)
df_filtered_sales_du = df_filtered_sales[(df_filtered_sales["du"] <= 252) & (df_filtered_sales["du"] > 0)]
display(df_filtered_sales_du.head())

,Chave,Cód. Cliente,Dt. Operação,Cód. Título,Cód. Corretora,Quantidade,Preço,Vencimento,DI,du
0,4721A,Cliente 1,2023-01-05,XPBR31,XPIN,4470,76.450544,2023-03-21,0.136781,51
1,19007A,Cliente 1,2023-01-05,XPBR31,XPIN,35530,76.440559,2023-03-21,0.136781,51
2,20955A,Cliente 1,2023-01-05,XPBR31,XPIN,5440,76.069360,2023-03-06,0.136704,40
3,30031A,Cliente 1,2023-01-05,XPBR31,XPIN,14560,76.079336,2023-03-06,0.136704,40
4,18899A,Cliente 1,2023-01-19,XPBR31,XPIN,41230,86.807599,2023-02-22,0.136585,22


In [ ]:
# create a list of dictionaries with the sales and their respective purchases that can be combined
from tqdm import tqdm
filtered_dfs = []
for index, row in tqdm(df_filtered_sales_du.head(100).iterrows()):
  available_purchases = df_filtered_purchases[
    (df_filtered_purchases['Cód. Cliente'] == row['Cód. Cliente']) &
    (df_filtered_purchases['Dt. Operação'] == row['Dt. Operação']) &
    (df_filtered_purchases['Cód. Corretora'] == row['Cód. Corretora']) &
    (df_filtered_purchases['Cód. Título'] == row['Cód. Título'])
  ]

  if not available_purchases.empty:
    filtered_dfs.append({
        'sale': row.copy(deep=True),
        'purchase': available_purchases.copy(deep=True)
    })

100it [00:29,  3.41it/s]


## Casos determinísticos

In [ ]:
df_merged_purchases = df_purchases.groupby(["Cód. Cliente", "Dt. Operação", "Cód. Corretora", "Cód. Título"]).agg({
    'Quantidade': 'sum',
    'Valor Líquido': 'sum',
    'Preço': 'mean'
}).reset_index()
display(df_merged_purchases)

,Cód. Cliente,Dt. Operação,Cód. Corretora,Cód. Título,Quantidade,Valor Líquido,Preço
0,Cliente 1,2023-01-02,CONV,ALPA4,1070,-15344.30,14.340467
1,Cliente 1,2023-01-02,CONV,AMER3,877,-7925.10,9.036602
2,Cliente 1,2023-01-02,CONV,AZUL4,473,-4875.46,10.307526
3,Cliente 1,2023-01-02,CONV,BBSE3,1011,-33781.96,33.414402
4,Cliente 1,2023-01-02,CONV,BEEF3,405,-5147.26,12.709284
...,...,...,...,...,...,...,...
85700,Cliente 9,2023-12-26,CONV,HBRE3,71,-418.34,5.892113
85701,Cliente 9,2023-12-26,CONV,SRNA3,1311,-13114.43,10.003379
85702,Cliente 9,2023-12-26,GOLD,VBBR3,349,-7748.76,22.186262
85703,Cliente 9,2023-12-26,MSDW,VBBR3,332,-7366.75,22.188763


In [ ]:
df_same_quantities_brute = pd.merge(df_merged_purchases, df_filtered_sales_du,
                                 on=['Cód. Cliente', 'Dt. Operação', 'Cód. Corretora', 'Cód. Título', 'Quantidade'],
                                 how='inner')
display(df_same_quantities_brute)

,Cód. Cliente,Dt. Operação,Cód. Corretora,Cód. Título,Quantidade,Valor Líquido,Preço_x,Chave,Preço_y,Vencimento,DI,du


# Aplicação do Agente

In [ ]:
from gym import Env
from gym.spaces import Discrete, Box, Dict
import numpy as np
import random
import os

In [ ]:
def calculate_ideal_price(sale_price, du, di):
  """Calculates the ideal average price for the combined purchases
     Uses the formula: pi=pv/(di+1)^du/252
     where:
     pi = ideal price
     pv = sale price
     du = business days
     di = DI tax of the day

  Args:
      sale_price (float): the mean price of the sale operation
      du (int): business days
      di (float): DI tax of the day

  Returns:
      float: ideal average price for the combined purchases
  """
  ideal_price = sale_price / ((di + 1) ** (du / 252))
  return ideal_price

In [ ]:
def calculate_current_price(old_price, prices_quantities):
    """Calculates the current average price for the combined purchases
  
      Args:
        old_price (float): The previous average price
        prices_quantities (list): A list of tuples containing the price and quantity of each agent action
        
      Returns:
        float: The new average price for the combined purchases
    """
    total_price = 0
    total_quantity = 0

    for price, quantity in prices_quantities:
        total_price += price * quantity
        total_quantity += quantity

    if total_quantity == 0:
      return old_price

    return total_price / total_quantity


In [ ]:
def calculate_reward(sale_price, ideal_price, current_price, old_price):
  """Calculates the reward for the agent based if the new average price is closer to the ideal average price than the old average price. Also, 
      if the new average price is higher than the sale price, the reward is -10, because this will result in a negative CDI.
    Args:
      sale_price (float): The price of the sale
      ideal_price (float): The average ideal price for the combined purchases
      current_price (float): The current average price
      old_price (float): The old average price
      
    Returns:
      int: The reward for the agent
  """

  old_distance = abs(ideal_price - old_price)
  new_distance = abs(ideal_price - current_price)
  reward = 0

  if current_price > sale_price:
    reward = -10
  if new_distance < old_distance:
    return 1 + reward

  elif new_distance == old_distance:
    return 0 + reward

  else:
    return -1 + reward

In [ ]:
def calculate_cdi(du, purchase_price, sale_price, di):
    """Calculates the CDI for the combined purchases
        Args:
            du (int): The number of business days between the sale operation date and the expiration date
            purchase_price (float): The average price of the combined purchases
            sale_price (float): The price of the sale
            di (float): The DI tax of the day
            
        Returns:
            float: The CDI for the combined purchases
    """
    rent = ((sale_price / purchase_price) - 1)
    annual_rent = ((1 + rent) ** (252 / du)) - 1

    cdi = (annual_rent / di) * 100

    return cdi

In [ ]:
# Hiperparams
max_steps = 100
learning_rate = 0.01
nb_steps = 100000
neurons_hidden_layer = 24
hidden_layer_activation = 'relu'

In [ ]:
observation_data = [
  "sale_price", 
  "sale_quantity", 
  "current_price", 
  "purchase_price", 
  "purchase_quantity", 
  "ideal_price"
]

In [ ]:
from copy import deepcopy

class Environment(Env):
    """ Implements the environment for an RL agent to learn how to combine purchases and sales to maximize the CDI around 100%

    Args:
        Env (object): OpenAI Gym environment as base class
    """
    def __init__(self):
        self.filtered_dfs = deepcopy(filtered_dfs)
        self.actions = [100, 75, 50, 25, 0, -25, -50, -75, -100]
        self.quantities_prices = [(0,0)]
        self.cdis = []
        self.best_cdis = []
        self.action_space = Discrete(len(self.actions))
        self.current_episode = 0
        self.current_step = 0
        self.max_steps = 0
        self.du = 0
        self.di = 0
        self.observation_space = Box(low=-np.inf, high=np.inf, shape=(len(observation_data),), dtype=np.float32)
        self.state = None

    def update_data(self, quantity):
        """Updates the quantities for the purchases and sales based on the agent's action

        Args:
            quantity (int): The percentage of the purchase quantity to be combined or discombined (positive or negative respectively)

        Returns:
            int: The actual quantity that can be combined or discombined
        """
        purchases = self.filtered_dfs[self.current_episode]['purchase']
        sale = self.filtered_dfs[self.current_episode]['sale']

        number_of_purchases = purchases.shape[0]
        current_purchase_index = self.current_step % number_of_purchases
        max_sale_qtd = filtered_dfs[self.current_episode]['sale']['Quantidade']
        current_purchase_qtd = purchases.iloc[current_purchase_index]["Quantidade"]
        combination_qtd = int((quantity / 100) * current_purchase_qtd)

        adjusted_quantity = min(combination_qtd, current_purchase_qtd)
        sale["Quantidade"] = self.state[1] - adjusted_quantity
        sale["Quantidade"] = max(sale["Quantidade"], 0)
        sale['Quantidade'] = min(max_sale_qtd, sale["Quantidade"])
        adjusted_quantity = max(sale['Quantidade'] - max_sale_qtd, adjusted_quantity)
        purchases.iloc[current_purchase_index, purchases.columns.get_loc("Quantidade")] = current_purchase_qtd - adjusted_quantity
        return adjusted_quantity

    def step(self, action):
        """Executes the action and updates the environment state by updating the quantities and prices of the purchases and sales, and calculating the reward and CDI
            The stop conditions are if the CDI is between 100 and 104 and the total sale quantity was used, or if the maximum number of steps is reached

        Args:
            action (_type_): The index of the action to be executed

        Returns:
            np.array: The new state of the environment
            int: The reward for the agent
            bool: Whether the episode is done or not
            dict: Additional information
        """
        done = False
        percent_to_act = self.actions[action]

        adjusted_quantity = self.update_data(percent_to_act)
        old_price = self.state[3]

        self.quantities_prices.append((self.state[2], adjusted_quantity))
        current_price = calculate_current_price(old_price, self.quantities_prices)

        reward = calculate_reward(self.state[0], self.state[5], current_price, old_price)

        self.current_step += 1
        self.state = self.__get_observation__()

        cdi = calculate_cdi(self.du, current_price, self.state[0], self.di)
        if (self.state[1] == 0
             and 100 <= cdi <= 104):
          reward += 1000
          done = True
          self.cdis.append(cdi)
          self.best_cdis.append(cdi)
          self.current_episode += 1

        elif self.current_step >= self.max_steps:
          reward += -10
          done = True
          self.cdis.append(cdi)
          self.current_episode += 1

        info = {}

        return self.state.reshape(len(observation_data)), reward, done, info

    def render(self, mode):
        pass

    def reset(self):
        """Resets the environment to the initial state, wich means going to the next sale

        Returns:
            np.array: The new state of the environment
        """
        self.current_step = 0
        self.quantities_prices = [(0, 0)]
        self.filtered_dfs = deepcopy(filtered_dfs)
        self.state = self.__get_observation__()
        return self.state.reshape(len(observation_data))

    def __get_observation__(self):
        """ Returns the observation data for the current state. It is updated on each step of the environment and episode
            It returns the sale price, sale quantity, current price, purchase price, purchase quantity and ideal price
            
            Returns:
                np.array: The observation data for the current state
        """
        sale = self.filtered_dfs[self.current_episode]['sale']
        purchases = self.filtered_dfs[self.current_episode]['purchase']

        number_of_purchases = purchases.shape[0]
        current_purchase_index  = self.current_step % number_of_purchases

        self.di = sale['DI']
        sale_price = sale['Preço']
        self.du = sale['du']

        ideal_price = calculate_ideal_price(sale_price, self.du, self.di)

        sale_qtd = sale['Quantidade']

        purchase_price = purchases.iloc[current_purchase_index]['Preço']
        purchase_qtd = purchases.iloc[current_purchase_index]['Quantidade']

        if self.current_step == 0:
          current_price = purchase_price
        else:
            current_price = calculate_current_price(self.state[3], self.quantities_prices)

        self.max_steps = number_of_purchases * max_steps

        return np.array([sale_price, sale_qtd, current_price, purchase_price, purchase_qtd, ideal_price]).reshape(len(observation_data),)

# Rede Neural e Treinamento

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from keras import __version__
tf.keras.__version__ = __version__

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

env = Environment()
states = env.observation_space.shape
actions = env.action_space.n

""" Creates a neural network model for the agent
    It has 3 hidden layers with 24 neurons each and a softmax output layer
"""
def build_model():
    model = Sequential()
    model.add(Flatten(input_shape=(1, len(observation_data))))
    model.add(Dense(neurons_hidden_layer, activation=hidden_layer_activation, input_shape=states))
    model.add(Dense(neurons_hidden_layer, activation=hidden_layer_activation))
    model.add(Dense(neurons_hidden_layer, activation=hidden_layer_activation))
    model.add(Dense(actions, activation='softmax'))
    return model

"""Creates an agent for the environment using the neural network model and DQN algorithm
"""
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-2)
    return dqn

In [ ]:
model = build_model()
model.summary()

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_37 (Flatten)        (None, 6)                 0         
                                                                 
 dense_148 (Dense)           (None, 24)                168       
                                                                 
 dense_149 (Dense)           (None, 24)                600       
                                                                 
 dense_150 (Dense)           (None, 24)                600       
                                                                 
 dense_151 (Dense)           (None, 9)                 225       
                                                                 
Total params: 1593 (6.22 KB)
Trainable params: 1593 (6.22 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

# train the agent
dqn = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate), metrics=['mae'])
dqn.fit(env, nb_steps=nb_steps, visualize=True, verbose=1, nb_max_episode_steps=None)
env.cdis

Training for 100000 steps ...
Interval 1 (0 steps performed)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


10000/10000 [==============================] - 158s 16ms/step - reward: -0.8036
2 episodes - episode_reward: 3986.000 [3983.000, 3989.000] - loss: 2.136 - mae: 0.283 - mean_q: 1.000

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 153s 15ms/step - reward: -1.9171
2 episodes - episode_reward: -19025.500 [-42079.000, 4028.000] - loss: 12.494 - mae: 0.491 - mean_q: 1.000

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 151s 15ms/step - reward: 0.5232
3 episodes - episode_reward: 2204.667 [-553.000, 3675.000] - loss: 8.696 - mae: 0.411 - mean_q: 1.000

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 157s 16ms/step - reward: -4.2181
2 episodes - episode_reward: -21548.500 [-46977.000, 3880.000] - loss: 8.966 - mae: 0.416 - mean_q: 1.000

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 161s 16ms/step - reward: 0.8489
2 episodes - episode_reward: 4590.000 [42

[93.27335097869438,
 92.74151095368765,
 -287.52982656515115,
 99.48893253135078,
 93.14141163237586,
 94.22948717358146,
 639.2198933988983,
 86.92505377802753,
 -75.05053172173048,
 85.01894294124807,
 64.6963782271836,
 268.27353129770876,
 276.78427196061944,
 79.18817431177378,
 259.30120848193565,
 106.07460488189562,
 101.86757681249263,
 -160.8279505108655,
 100.09826528381605,
 60.80316875185614,
 134.60324467665174,
 -278.20864516229034,
 95.03655003759037,
 103.49114936081803]

In [ ]:
env.best_cdis

[101.86757681249263, 100.09826528381605, 103.49114936081803]

In [ ]:
# test the agent
env.current_episode = 0
env.cdis = []
env.best_cdis = []
scores = dqn.test(env, nb_episodes=len(filtered_dfs), visualize=False)
print(np.mean(scores.history['episode_reward']))
print(env.cdis)
print(env.best_cdis)

print(f"Acertos: {(len(env.best_cdis) / len(env.cdis)) * 100}")

[]
Testing for 100 episodes ...
Episode 1: reward: 3809.000, steps: 4200
Episode 2: reward: 3809.000, steps: 4200
Episode 3: reward: 3809.000, steps: 4200
Episode 4: reward: 3809.000, steps: 4200
Episode 5: reward: 3689.000, steps: 3900
Episode 6: reward: 3689.000, steps: 3900
Episode 7: reward: 3889.000, steps: 3900
Episode 8: reward: 3889.000, steps: 3900
Episode 9: reward: 4487.000, steps: 5100
Episode 10: reward: 3693.000, steps: 5100
Episode 11: reward: 3693.000, steps: 5100
Episode 12: reward: 3693.000, steps: 5100
Episode 13: reward: 3693.000, steps: 5100
Episode 14: reward: 4487.000, steps: 5100
Episode 15: reward: 3693.000, steps: 5100
Episode 16: reward: 4882.000, steps: 5300
Episode 17: reward: 5082.000, steps: 5300
Episode 18: reward: 3791.000, steps: 4000
Episode 19: reward: 1000.000, steps: 1
Episode 20: reward: 791.000, steps: 6000
Episode 21: reward: 991.000, steps: 6000
Episode 22: reward: 4885.000, steps: 4900
Episode 23: reward: 4885.000, steps: 4900
Episode 24: rewa

In [ ]:
print(env.cdis)
print(env.best_cdis)

[93.27335097868902, 92.74151095368133, 94.58874112988532, 95.27147246735291, 93.1414116324339, 94.22948717362942, 87.56377353880079, 86.92505377805631, 75.78048657456391, 272.57657349098696, 264.21468446502627, 268.2735312977764, 276.7842719606877, 79.18817431180402, 259.30120848197225, 106.07460488185541, 104.66280400072263, 99.09688405771672, 100.09826528381393, 182.2925610905334, 181.36138823878153, 94.4925252810829, 95.03655003760912, 103.49114936081803, 89.29588170640183, 88.97047849943944, 88.03256579558861, 90.42772736174479, 94.04466232755905, 93.81330128017055, 94.73569848878228, 111.71648815583394, 110.96271145641698, 83.43028388846223, 84.29871340793821, 95.95601633582235, 103.94311995179689, 103.82438057584396, 104.32244056680555, 95.83729635649438, 97.26861316990674, 98.41232014101635, 96.82428656442487, 94.86542186845949, 94.04934163424116, 95.25203004531323, 103.953083359915, 7.319336555330248, 7.31934299357002, 5.866070618708811, 5.866057243955714, 94.66088851016458, 93